In [ ]:
# ============================================================
# B2B Sales & CRM Analytics Dashboard — Google Colab Script
# Senior RevOps Data Analyst | Complete Single-Cell Script
# ============================================================

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# ─────────────────────────────────────────────
# STEP 1: GENERATE REALISTIC B2B SALES DATA
# ─────────────────────────────────────────────
np.random.seed(42)
N = 2500

start_date = pd.Timestamp('2024-01-01')
end_date   = pd.Timestamp('2024-12-31')
date_range_days = (end_date - start_date).days

# Created dates spread over 12 months
created_dates = [start_date + pd.Timedelta(days=int(d))
                 for d in np.random.randint(0, date_range_days, N)]

# Sales reps
reps = np.random.choice(['Ahmed', 'Sara', 'Omar', 'Nour', 'Khaled'], size=N,
                         p=[0.22, 0.21, 0.20, 0.19, 0.18])

# Deal values
deal_values = np.random.uniform(20_000, 500_000, N).round(2)

# Stage distribution — realistic funnel (Closed Lost > Closed Won)
stage_probs = [0.30, 0.22, 0.18, 0.10, 0.10, 0.10]   # Lead, Discovery, Proposal, Negotiation, Closed Won, Closed Lost
stages = np.random.choice(
    ['Lead', 'Discovery', 'Proposal', 'Negotiation', 'Closed Won', 'Closed Lost'],
    size=N, p=stage_probs
)

# Close dates: only for Closed Won / Closed Lost
close_dates = []
for i in range(N):
    if stages[i] in ('Closed Won', 'Closed Lost'):
        days_to_close = np.random.randint(10, 91)
        close_dates.append(created_dates[i] + pd.Timedelta(days=days_to_close))
    else:
        close_dates.append(pd.NaT)

df = pd.DataFrame({
    'Deal_ID':        [f'DEAL-{str(i+1).zfill(4)}' for i in range(N)],
    'Created_Date':   created_dates,
    'Sales_Rep':      reps,
    'Deal_Value_EGP': deal_values,
    'Stage':          stages,
    'Close_Date':     close_dates
})

df['Created_Date'] = pd.to_datetime(df['Created_Date'])
df['Close_Date']   = pd.to_datetime(df['Close_Date'])

print(f"✅ Dataset generated: {len(df):,} deals")
print(df['Stage'].value_counts())

# ─────────────────────────────────────────────
# STEP 2: CALCULATE CRM KPIs
# ─────────────────────────────────────────────
won  = df[df['Stage'] == 'Closed Won']
lost = df[df['Stage'] == 'Closed Lost']
open_deals = df[~df['Stage'].isin(['Closed Won', 'Closed Lost'])]

total_revenue     = won['Deal_Value_EGP'].sum()
win_rate          = len(won) / (len(won) + len(lost)) * 100
avg_sales_cycle   = (won['Close_Date'] - won['Created_Date']).dt.days.mean()
open_pipeline_val = open_deals['Deal_Value_EGP'].sum()

print(f"\n📊 KPIs")
print(f"  Closed Won Revenue : EGP {total_revenue:,.0f}")
print(f"  Win Rate           : {win_rate:.1f}%")
print(f"  Avg Sales Cycle    : {avg_sales_cycle:.1f} days")
print(f"  Open Pipeline      : EGP {open_pipeline_val:,.0f}")

# ─────────────────────────────────────────────
# STEP 3: CREATE 3 INDEPENDENT PLOTLY FIGURES
# ─────────────────────────────────────────────

# ── Color palette ──────────────────────────
C_INDIGO    = '#4F46E5'
C_SLATE     = '#64748B'
C_BLUE      = '#2563EB'
C_LIGHT_BLU = '#93C5FD'
C_BG        = '#111827'
C_CARD      = '#1F2937'
C_TEXT      = '#F9FAFB'
C_SUBTEXT   = '#9CA3AF'

LAYOUT_BASE = dict(
    paper_bgcolor=C_CARD,
    plot_bgcolor=C_CARD,
    font=dict(family='Inter, Segoe UI, sans-serif', color=C_TEXT, size=13),
    margin=dict(l=40, r=40, t=60, b=40),
    legend=dict(bgcolor='rgba(0,0,0,0)', font=dict(color=C_TEXT))
)

# ── Fig 1: Sales Funnel ────────────────────
funnel_stages = ['Lead', 'Discovery', 'Proposal', 'Negotiation', 'Closed Won']
funnel_counts = [int(df[df['Stage'] == s].shape[0]) for s in funnel_stages]

fig_funnel = go.Figure(go.Funnel(
    y=funnel_stages,
    x=funnel_counts,
    textinfo='value+percent initial',
    marker=dict(color=[C_INDIGO, C_BLUE, '#1D4ED8', '#1E40AF', '#16A34A']),
    connector=dict(line=dict(color=C_SLATE, width=1))
))
fig_funnel.update_layout(
    title=dict(text='Sales Funnel Conversion', font=dict(size=16, color=C_TEXT)),
    **LAYOUT_BASE
)

# ── Fig 2: Sales Rep Performance ──────────
rep_won = (won.groupby('Sales_Rep')['Deal_Value_EGP'].sum() / 1e6).round(2)
rep_win_rate = (
    won.groupby('Sales_Rep').size() /
    (won.groupby('Sales_Rep').size() + lost.groupby('Sales_Rep').size())
    * 100
).round(1)

all_reps = sorted(df['Sales_Rep'].unique())
rev_vals  = [rep_won.get(r, 0)      for r in all_reps]
wr_vals   = [rep_win_rate.get(r, 0) for r in all_reps]

fig_reps = make_subplots(specs=[[{'secondary_y': True}]])

fig_reps.add_trace(go.Bar(
    name='Revenue (M EGP)',
    x=all_reps, y=rev_vals,
    marker_color=C_INDIGO,
    text=[f'{v:.2f}M' for v in rev_vals],
    textposition='outside',
    textfont=dict(color=C_TEXT)
), secondary_y=False)

fig_reps.add_trace(go.Bar(
    name='Win Rate %',
    x=all_reps, y=wr_vals,
    marker_color=C_BLUE,
    text=[f'{v:.1f}%' for v in wr_vals],
    textposition='outside',
    textfont=dict(color=C_TEXT)
), secondary_y=True)

fig_reps.update_layout(
    title=dict(text='Sales Rep Performance (Revenue vs Win Rate)', font=dict(size=16, color=C_TEXT)),
    barmode='group',
    paper_bgcolor=C_CARD,
    plot_bgcolor=C_CARD,
    font=dict(family='Inter, Segoe UI, sans-serif', color=C_TEXT, size=13),
    margin=dict(l=40, r=40, t=60, b=40),
    legend=dict(bgcolor='rgba(0,0,0,0)', font=dict(color=C_TEXT))
)
fig_reps.update_yaxes(title_text='Revenue (M EGP)', secondary_y=False,
                       gridcolor='#374151', color=C_TEXT)
fig_reps.update_yaxes(title_text='Win Rate %', secondary_y=True,
                       gridcolor='#374151', color=C_TEXT)
fig_reps.update_xaxes(gridcolor='#374151', color=C_TEXT)

# ── Fig 3: Monthly Revenue Trend ──────────
won_copy = won.copy()
won_copy['Month'] = won_copy['Close_Date'].dt.to_period('M')
monthly_rev = won_copy.groupby('Month')['Deal_Value_EGP'].sum().reset_index()
monthly_rev['Month_str'] = monthly_rev['Month'].astype(str)
monthly_rev['Revenue_M'] = (monthly_rev['Deal_Value_EGP'] / 1e6).round(2)

fig_trend = go.Figure()
fig_trend.add_trace(go.Scatter(
    x=monthly_rev['Month_str'],
    y=monthly_rev['Revenue_M'],
    mode='lines+markers',
    name='Monthly Revenue',
    line=dict(color=C_INDIGO, width=3),
    marker=dict(size=8, color=C_BLUE, line=dict(color=C_LIGHT_BLU, width=2)),
    fill='tozeroy',
    fillcolor='rgba(79,70,229,0.15)'
))
fig_trend.update_layout(
    title=dict(text='Monthly Revenue Growth', font=dict(size=16, color=C_TEXT)),
    xaxis=dict(title='Month', gridcolor='#374151', color=C_TEXT),
    yaxis=dict(title='Revenue (M EGP)', gridcolor='#374151', color=C_TEXT),
    **LAYOUT_BASE
)

print("✅ All 3 Plotly figures created.")

# ─────────────────────────────────────────────
# STEP 4: BUILD PURE HTML/CSS DASHBOARD
# ─────────────────────────────────────────────

funnel_html = fig_funnel.to_html(full_html=False, include_plotlyjs=False)
reps_html   = fig_reps.to_html(full_html=False, include_plotlyjs=False)
trend_html  = fig_trend.to_html(full_html=False, include_plotlyjs=False)

# KPI formatting helpers
def fmt_egp(val):
    if val >= 1e6:
        return f"EGP {val/1e6:.2f}M"
    return f"EGP {val:,.0f}"

kpi_revenue   = fmt_egp(total_revenue)
kpi_win_rate  = f"{win_rate:.1f}%"
kpi_cycle     = f"{avg_sales_cycle:.1f} Days"
kpi_pipeline  = fmt_egp(open_pipeline_val)

html_dashboard = f"""<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1.0"/>
  <title>B2B Sales & CRM Analytics Dashboard</title>
  <script src="https://cdn.plot.ly/plotly-2.27.0.min.js"></script>
  <style>
    *, *::before, *::after {{ box-sizing: border-box; margin: 0; padding: 0; }}

    body {{
      background: #111827;
      font-family: 'Inter', 'Segoe UI', sans-serif;
      color: #F9FAFB;
      min-height: 100vh;
      padding: 24px;
    }}

    /* ── Header ── */
    .header {{
      display: flex;
      align-items: center;
      justify-content: space-between;
      margin-bottom: 28px;
    }}
    .header h1 {{
      font-size: 1.6rem;
      font-weight: 700;
      background: linear-gradient(90deg, #818CF8, #60A5FA);
      -webkit-background-clip: text;
      -webkit-text-fill-color: transparent;
      letter-spacing: -0.5px;
    }}
    .header .badge {{
      background: #1F2937;
      border: 1px solid #374151;
      border-radius: 20px;
      padding: 6px 16px;
      font-size: 0.78rem;
      color: #9CA3AF;
    }}

    /* ── KPI Grid ── */
    .kpi-grid {{
      display: grid;
      grid-template-columns: repeat(4, 1fr);
      gap: 16px;
      margin-bottom: 24px;
    }}
    .kpi-card {{
      background: #1F2937;
      border: 1px solid #374151;
      border-radius: 12px;
      padding: 20px 22px;
      position: relative;
      overflow: hidden;
      transition: transform 0.2s;
    }}
    .kpi-card:hover {{ transform: translateY(-2px); }}
    .kpi-card::before {{
      content: '';
      position: absolute;
      top: 0; left: 0; right: 0;
      height: 3px;
    }}
    .kpi-card.indigo::before  {{ background: linear-gradient(90deg, #4F46E5, #818CF8); }}
    .kpi-card.blue::before    {{ background: linear-gradient(90deg, #2563EB, #60A5FA); }}
    .kpi-card.green::before   {{ background: linear-gradient(90deg, #16A34A, #4ADE80); }}
    .kpi-card.slate::before   {{ background: linear-gradient(90deg, #64748B, #94A3B8); }}

    .kpi-label {{
      font-size: 0.75rem;
      font-weight: 600;
      text-transform: uppercase;
      letter-spacing: 0.08em;
      color: #9CA3AF;
      margin-bottom: 10px;
    }}
    .kpi-value {{
      font-size: 1.65rem;
      font-weight: 700;
      color: #F9FAFB;
      line-height: 1.1;
    }}
    .kpi-sub {{
      font-size: 0.78rem;
      color: #6B7280;
      margin-top: 6px;
    }}

    /* ── Charts Grid ── */
    .charts-row {{
      display: grid;
      grid-template-columns: 1fr 1fr;
      gap: 16px;
      margin-bottom: 16px;
    }}
    .chart-card {{
      background: #1F2937;
      border: 1px solid #374151;
      border-radius: 12px;
      padding: 4px;
      overflow: hidden;
    }}
    .chart-card.full-width {{
      grid-column: 1 / -1;
    }}

    /* ── Footer ── */
    .footer {{
      text-align: center;
      margin-top: 20px;
      font-size: 0.75rem;
      color: #4B5563;
    }}

    /* Ensure Plotly charts fill their containers */
    .chart-card .plotly-graph-div {{
      width: 100% !important;
    }}
  </style>
</head>
<body>

  <!-- Header -->
  <div class="header">
    <h1>📊 B2B Sales & CRM Analytics Dashboard</h1>
    <span class="badge">FY 2024 · 2,500 Deals · EGP</span>
  </div>

  <!-- KPI Cards -->
  <div class="kpi-grid">
    <div class="kpi-card indigo">
      <div class="kpi-label">💰 Closed Won Revenue</div>
      <div class="kpi-value">{kpi_revenue}</div>
      <div class="kpi-sub">Total recognized revenue</div>
    </div>
    <div class="kpi-card green">
      <div class="kpi-label">🏆 Win Rate</div>
      <div class="kpi-value">{kpi_win_rate}</div>
      <div class="kpi-sub">Won vs (Won + Lost)</div>
    </div>
    <div class="kpi-card blue">
      <div class="kpi-label">⏱️ Avg Sales Cycle</div>
      <div class="kpi-value">{kpi_cycle}</div>
      <div class="kpi-sub">Closed Won deals only</div>
    </div>
    <div class="kpi-card slate">
      <div class="kpi-label">🔄 Open Pipeline</div>
      <div class="kpi-value">{kpi_pipeline}</div>
      <div class="kpi-sub">Excluding closed deals</div>
    </div>
  </div>

  <!-- Charts Row 1: Funnel + Reps -->
  <div class="charts-row">
    <div class="chart-card">
      {funnel_html}
    </div>
    <div class="chart-card">
      {reps_html}
    </div>
  </div>

  <!-- Charts Row 2: Trend (full width) -->
  <div class="charts-row">
    <div class="chart-card full-width">
      {trend_html}
    </div>
  </div>

  <!-- Footer -->
  <div class="footer">
    Generated by Senior RevOps Data Analyst · Powered by Plotly &amp; Python · 2024
  </div>

</body>
</html>"""

# ─────────────────────────────────────────────
# STEP 5: EXPORT TO HTML FILE
# ─────────────────────────────────────────────
output_file = 'B2B_Sales_CRM_Dashboard.html'
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(html_dashboard)

print(f"\n✅ Dashboard saved → {output_file}")
print("   Open it in any browser to view the interactive dashboard.")